In [ ]:
##################### Requirements ######################
!pip install openai
!pip install llama_index
!pip install llama-index-embeddings-huggingface
!pip install torch sentence-transformers

In [ ]:
import os
import openai

from llama_index.core import (
     SimpleDirectoryReader,
     load_index_from_storage,
     StorageContext,
     VectorStoreIndex,
     ServiceContext,
     PromptTemplate,
     get_response_synthesizer,
)

from llama_index.core.text_splitter import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core.postprocessor.sbert_rerank import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

# global
from llama_index.core import Settings

In [ ]:
import os
import openai
from google.colab import userdata

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key= userdata.get('OPENAI_API_KEY')


In [ ]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()
from google.colab import userdata

import nest_asyncio
nest_asyncio.apply()

# bring in deps
from llama_cloud_services import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    api_key=userdata.get('LLAMA_CLOUD_API_KEY')
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser, ".docx": parser}
documents = SimpleDirectoryReader(input_dir='data', file_extractor=file_extractor).load_data()
print(documents)

Started parsing the file under job_id 3cf71167-3706-46d1-9cf4-0b0fe99d8d9a
.Started parsing the file under job_id ac2ca4dc-8101-420f-839d-7bbb053f28b7
.Started parsing the file under job_id 389b6b3a-f72a-496f-86eb-df320ad54a40
[Document(id_='d20c0901-36ec-47fa-b1fc-1ac6d27d961c', embedding=None, metadata={'file_path': '/content/data/4. Capstone HR Search Criteria and Forms_DO NOT SHARE-POST-PUBLISH.pdf', 'file_name': '4. Capstone HR Search Criteria and Forms_DO NOT SHARE-POST-PUBLISH.pdf', 'file_type': 'application/pdf', 'file_size': 164017, 'creation_date': '2025-03-08', 'last_modified_date': '2025-03-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, 

In [ ]:
from llama_index.core.text_splitter import SentenceSplitter

def create_nodes(documents, chunk_size=500, chunk_overlap=100):
  node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  nodes = node_parser.get_nodes_from_documents(documents)
  return nodes

nodes1 = create_nodes(documents)
print(len(nodes1)) # 29

86


In [ ]:
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.7)
Settings.embed_model = "local:BAAI/bge-large-en-v1.5"
print(f"Models Loaded")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Models Loaded


In [ ]:
# Load or create the index
try:
    from llama_index.core import load_index_from_storage, StorageContext
    storage_context = StorageContext.from_defaults(persist_dir="/content/dd")
    sentence_index = load_index_from_storage(nodes=nodes1,storage_context=storage_context)
except:
    sentence_index = VectorStoreIndex(nodes1)
    sentence_index.storage_context.persist("/content/dd")

In [ ]:
# from llama_index.core.indices.postprocessor import SentenceTransformerRerank

from llama_index.core.postprocessor.sbert_rerank import SentenceTransformerRerank
bge_reranker_large= SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "If the context information does not contain an answer to the query, "
            "respond with \"No information\".\n"
            "Query: {query_str}\n"
            "Answer: "
        )
qa_prompt = PromptTemplate(qa_prompt_tmpl)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine


retriever = VectorIndexRetriever(
    index=sentence_index,
    similarity_top_k=5,
)

response_synthesizer = get_response_synthesizer(
    # service_context=service_context,
    text_qa_template=qa_prompt,
    response_mode="refine",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[ bge_reranker_large ]
)

In [ ]:
responseranker = query_engine.query("What are the Deliverables for the Identifying Capstone Officials")
print(str(responseranker))
print("--------")

for i in range(len(responseranker.source_nodes)):
    print(responseranker.source_nodes[i])

The Deliverables for the Identifying Capstone Officials include:

- Step by step description of identification method(s):
  - Approach for identifying Capstone officials
  - Approaches for identifying individuals in Acting Capstone positions specifically utilizing:
--------
Node ID: 20179d81-8915-45e9-9769-4ef901197739
Text: manual reports be run by HR in Insight identifying Capstone
officials. The search criteria used is included in the provided
information.  USDA, and other Federal agencies, also have temporary
Capstone officials. These are individuals who are typically tenured
employees, who temporarily step into a Capstone position to ensure the
associated dutie...
Score:  0.993

Node ID: 8040801e-2ff1-4fd5-b664-86c007931f7c
Text: # Problem Statement  Organization: The General Counsel’s Office
of Information Affairs (OIA) Records Management Division, United
States Department of Agriculture (USDA)  Challenge Name: Identifying
Capstone Officials  Category: AI Challenge  # Organizatio